### EDA with DataFrames and Spark SQL

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

23/07/15 11:14:22 WARN Utils: Your hostname, bhajji resolves to a loopback address: 127.0.1.1; using 192.168.2.24 instead (on interface wlp2s0)
23/07/15 11:14:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/07/15 11:14:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/15 11:14:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
json_df2_path = "./Data/utilization.json"
df_util = spark.read.format("json").load(json_df2_path)

In [3]:
df_util.show()

+---------------+-------------------+-----------+---------+-------------+
|cpu_utilization|     event_datetime|free_memory|server_id|session_count|
+---------------+-------------------+-----------+---------+-------------+
|           0.57|03/05/2019 08:06:14|       0.51|      100|           47|
|           0.47|03/05/2019 08:11:14|       0.62|      100|           43|
|           0.56|03/05/2019 08:16:14|       0.57|      100|           62|
|           0.57|03/05/2019 08:21:14|       0.56|      100|           50|
|           0.35|03/05/2019 08:26:14|       0.46|      100|           43|
|           0.41|03/05/2019 08:31:14|       0.58|      100|           48|
|           0.57|03/05/2019 08:36:14|       0.35|      100|           58|
|           0.41|03/05/2019 08:41:14|        0.4|      100|           58|
|           0.53|03/05/2019 08:46:14|       0.35|      100|           62|
|           0.51|03/05/2019 08:51:14|        0.6|      100|           45|
|           0.32|03/05/2019 08:56:14| 

In [4]:
df_util.createOrReplaceTempView("utilization")

In [5]:
df_util.count()

500000

In [6]:
df_util.describe().show()

+-------+-------------------+-------------------+-------------------+------------------+------------------+
|summary|    cpu_utilization|     event_datetime|        free_memory|         server_id|     session_count|
+-------+-------------------+-------------------+-------------------+------------------+------------------+
|  count|             500000|             500000|             500000|            500000|            500000|
|   mean| 0.6205177400000106|               null| 0.3791280999999971|             124.5|          69.59616|
| stddev|0.15875173872912937|               null|0.15830931278376237|14.430884120552962|14.850676696352718|
|    min|               0.22|03/05/2019 08:06:14|                0.0|               100|                32|
|    max|                1.0|04/09/2019 01:22:46|               0.78|               149|               105|
+-------+-------------------+-------------------+-------------------+------------------+------------------+



In [7]:
df_util.describe('cpu_utilization','free_memory').show()

+-------+-------------------+-------------------+
|summary|    cpu_utilization|        free_memory|
+-------+-------------------+-------------------+
|  count|             500000|             500000|
|   mean| 0.6205177400000106| 0.3791280999999971|
| stddev|0.15875173872912937|0.15830931278376237|
|    min|               0.22|                0.0|
|    max|                1.0|               0.78|
+-------+-------------------+-------------------+



In [8]:
df_util.stat.corr('cpu_utilization','free_memory')

-0.47047715730807543

In [9]:
df_util.stat.corr('session_count','free_memory')

-0.5008320848876592

In [10]:
df_util.stat.freqItems(('server_id','session_count')).show()

+--------------------+-----------------------+
| server_id_freqItems|session_count_freqItems|
+--------------------+-----------------------+
|[146, 137, 101, 1...|   [92, 101, 83, 104...|
+--------------------+-----------------------+



In [11]:
df_util_sample = df_util.sample(fraction=0.05, withReplacement=False)
df_util_sample.count()

24778

In [12]:
spark.sql('SELECT min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) FROM utilization').show()

+--------------------+--------------------+-----------------------+
|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+--------------------+--------------------+-----------------------+
|                0.22|                 1.0|    0.15875173872912937|
+--------------------+--------------------+-----------------------+



In [13]:
spark.sql('SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id').show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353687|
|      110|                0.35|                0.75|    0.11533251724450215|
|      107|                0.45|                0.85|    0.11597417369783877|
|      103|                0.56|                0.96|    0.11617507884178278|
|      104|                0.51|                0.91|    0.11521679513850511|
|      106|                0.22|                0.62|    0.11531539914568233|
|      111|                0.36|                0.76|    0.11530221569464506|
|      100|                0.27|                0.67|     0.1152264191787964|
|      105|                0.29|                0.69|    0.11510

In [14]:
spark.sql('SELECT server_id, FLOOR(cpu_utilization*100/10) bucket FROM utilization').show()

+---------+------+
|server_id|bucket|
+---------+------+
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     4|
|      100|     5|
|      100|     4|
|      100|     5|
|      100|     5|
|      100|     3|
|      100|     6|
|      100|     6|
|      100|     5|
|      100|     2|
|      100|     4|
|      100|     4|
|      100|     6|
|      100|     4|
|      100|     5|
+---------+------+
only showing top 20 rows



### EDA with Time Series Data

In [15]:
spark.sql('SELECT server_id, min(cpu_utilization), max(cpu_utilization), stddev(cpu_utilization) \
           FROM utilization \
           GROUP BY server_id').show()

+---------+--------------------+--------------------+-----------------------+
|server_id|min(cpu_utilization)|max(cpu_utilization)|stddev(cpu_utilization)|
+---------+--------------------+--------------------+-----------------------+
|      112|                0.52|                0.92|    0.11528867845082576|
|      113|                0.58|                0.98|    0.11544345150353687|
|      110|                0.35|                0.75|    0.11533251724450215|
|      107|                0.45|                0.85|    0.11597417369783877|
|      103|                0.56|                0.96|    0.11617507884178278|
|      104|                0.51|                0.91|    0.11521679513850511|
|      106|                0.22|                0.62|    0.11531539914568233|
|      111|                0.36|                0.76|    0.11530221569464506|
|      100|                0.27|                0.67|     0.1152264191787964|
|      105|                0.29|                0.69|    0.11510

In [16]:
sql_window = "SELECT event_datetime, server_id, cpu_utilization,  \
         avg(cpu_utilization) OVER (PARTITION BY server_id) avg_server_util \
FROM  \
      utilization"

In [17]:
spark.sql(sql_window).show()

+-------------------+---------+---------------+------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|
+-------------------+---------+---------------+------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|
|03/05/2019 08:46:31|      110|           0.37|0.5537749999999892|
|03/05/2019 08:51:31|      110|            0.7|0.5537749999999892|
|03/05/2019 08:56:31|      110|           0.67|0.5537749999999892|
|03/05/2019 09:01:31|      110|           0.56|0.5537749999999

In [18]:
sql_window2 = "SELECT event_datetime, server_id, cpu_utilization,  \
         avg(cpu_utilization) OVER (PARTITION BY server_id) avg_server_util, \
         cpu_utilization - avg(cpu_utilization) OVER (PARTITION BY server_id) delta_server_util \
         FROM utilization"

spark.sql(sql_window2).show()

+-------------------+---------+---------------+------------------+--------------------+
|     event_datetime|server_id|cpu_utilization|   avg_server_util|   delta_server_util|
+-------------------+---------+---------------+------------------+--------------------+
|03/05/2019 08:06:31|      110|           0.68|0.5537749999999892|  0.1262250000000108|
|03/05/2019 08:11:31|      110|           0.58|0.5537749999999892|0.026225000000010712|
|03/05/2019 08:16:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:21:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:26:31|      110|           0.63|0.5537749999999892| 0.07622500000001076|
|03/05/2019 08:31:31|      110|           0.71|0.5537749999999892| 0.15622500000001072|
|03/05/2019 08:36:31|      110|           0.67|0.5537749999999892| 0.11622500000001079|
|03/05/2019 08:41:31|      110|           0.55|0.5537749999999892|-0.00377499999998...|
|03/05/2019 08:46:31|      110| 

In [19]:
sql_window3 = "SELECT event_datetime, server_id, cpu_utilization,  \
                      avg(cpu_utilization) OVER (PARTITION BY server_id ORDER BY event_datetime \
                                    ROWS BETWEEN 1 PRECEDING AND 1 FOLLOWING) avg_server_util \
                FROM  \
                      utilization"

spark.sql(sql_window3).show()

+-------------------+---------+---------------+-------------------+
|     event_datetime|server_id|cpu_utilization|    avg_server_util|
+-------------------+---------+---------------+-------------------+
|03/05/2019 08:06:31|      110|           0.68|               0.63|
|03/05/2019 08:11:31|      110|           0.58| 0.6033333333333334|
|03/05/2019 08:16:31|      110|           0.55| 0.5866666666666666|
|03/05/2019 08:21:31|      110|           0.63| 0.6033333333333334|
|03/05/2019 08:26:31|      110|           0.63| 0.6566666666666666|
|03/05/2019 08:31:31|      110|           0.71| 0.6699999999999999|
|03/05/2019 08:36:31|      110|           0.67| 0.6433333333333333|
|03/05/2019 08:41:31|      110|           0.55| 0.5300000000000001|
|03/05/2019 08:46:31|      110|           0.37|               0.54|
|03/05/2019 08:51:31|      110|            0.7|               0.58|
|03/05/2019 08:56:31|      110|           0.67| 0.6433333333333334|
|03/05/2019 09:01:31|      110|           0.56| 